In [48]:
def generate_data(args):
    """
    This implements the generative process of [1], but using larger feature and
    latent spaces ([1] assumes ``feature_dim=1`` and ``latent_dim=5``).
    """
    z = dist.Bernoulli(0.5).sample([args.num_data])
    x = dist.Normal(z, 5 * z + 3 * (1 - z)).sample([args.feature_dim]).t()
    t = dist.Bernoulli(0.75 * z + 0.25 * (1 - z)).sample()
    y = dist.Bernoulli(logits=3 * (z + 2 * (2 * t - 2))).sample()

    # Compute true ite for evaluation (via Monte Carlo approximation).
    t0_t1 = torch.tensor([[0.0], [1.0]])
    y_t0, y_t1 = dist.Bernoulli(logits=3 * (z + 2 * (2 * t0_t1 - 2))).mean
    true_ite = y_t1 - y_t0
    return x, t, y, true_ite

In [51]:
import pyro
import argparse
import logging
import torch
import pyro.distributions as dist
import pandas as pd

if __name__ == "__main__":
    assert pyro.__version__.startswith("1.8.1")
    parser = argparse.ArgumentParser(
        description="dataset generator"
    )
    parser.add_argument('-f')
    parser.add_argument("--num-data", default=30000, type=int)
    parser.add_argument("--feature-dim", default=10, type=int)
    args = parser.parse_args()

In [52]:
x, t, y, true_ite = generate_data(args)
x, t, y, true_ite = pd.DataFrame(x.numpy()),  pd.DataFrame(t.numpy()),  pd.DataFrame(y.numpy()),  pd.DataFrame(true_ite.numpy())
data = pd.concat((x,t,y,true_ite), axis = 1)
data.columns = [['x0', 'x1','x2','x3','x4','x5','x6','x7','x8','x9','treatment','y','true_ite']]
print(data.shape)
data.head()
data.to_csv('10_dim_x_y_ite_30000_sample.csv')

(30000, 13)
